In [ ]:
import os
import sys
import glob
import json 
import pprint

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
MODEL_TRAINING_OUTPUT_BASE_DIR = '/media/data/local/corn/out/19_12_21/'

In [ ]:
dir_names = [name for name in os.listdir(MODEL_TRAINING_OUTPUT_BASE_DIR) if os.path.isdir(os.path.join(MODEL_TRAINING_OUTPUT_BASE_DIR, name))]
results = {}


for dir_name in dir_names:
    if not dir_name.startswith('model_'):
        print(f'Skipping directory {dir_name}')
        continue
    
    dir_path = os.path.join(MODEL_TRAINING_OUTPUT_BASE_DIR, dir_name)
    model_result_files = glob.glob(os.path.join(dir_path, 'model_*_result.json'))
    if len(model_result_files) != 1:
        print(f'Invalid directory results in {dir_name}')
        continue
        
    model_result_file_path = model_result_files[0]
    
    def remove_prefix(text, prefix):
        return text[text.startswith(prefix) and len(prefix):]

    model_name = remove_prefix(dir_name, 'model_')
    with open(model_result_file_path, 'rt') as file:
        results[model_name] = json.load(file)
        

In [ ]:
test_results_metrics = {
    'fscore': {},
    'iou_score': {},
    'IoU_Class1': {},
    'accuracy': {},
    'dice_loss': {},
}

for model_name, result in results.items():
    for metric_name in test_results_metrics.keys():
        test_results_metrics[metric_name][model_name] = result['test_log'][metric_name]
        


In [ ]:
# pprint.pprint(test_results_metrics)

for metric_name in test_results_metrics.keys():
    data_dict = test_results_metrics[metric_name]
    data_dict = dict(sorted(data_dict.items(), key=lambda item: item[1], reverse=True))
    keys = data_dict.keys()
    values = data_dict.values()
    plt.bar(keys, values)
    plt.xticks(rotation='vertical')
    plt.ylim([min(data_dict.values())-0.1, max(data_dict.values())+0.1])
    plt.title(metric_name)
    plt.grid()
    plt.show()
    